In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI：Vertex AI 迁移：使用预构建训练容器的自定义 XGBoost 模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-xgboost-prebuilt-container.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-xgboost-prebuilt-container.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-custom-xgboost-prebuilt-container.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                      
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署用于批量预测的自定义表格分类XGBoost模型。

了解更多关于[迁移到Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的信息。

### 目标

在本教程中，您将学习如何使用`Vertex AI Training`创建一个自定义训练模型，并使用`Vertex AI Batch Prediction`对训练模型进行批量预测。

您将学习如何使用Vertex AI SDK for Python在Docker容器中从Python脚本创建一个自定义训练模型，然后通过发送数据对部署的模型进行预测。

本教程使用以下谷歌云ML服务：

- `Vertex AI Training`
- `Vertex AI Batch Prediction`
- `Vertex AI Model`资源
- `Vertex AI Endpoint`资源

执行的步骤包括：

- 创建一个用于训练scikit-learn模型的`Vertex AI`自定义作业。
- 将训练模型工件上传为`Model`资源。
- 进行批量预测。
- 部署模型到一个终端。
- 进行在线预测。

数据集

本教程使用的数据集是[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)中的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview) 。这个数据集不需要任何特征工程。在本教程中您将使用的数据集版本存储在一个公共云存储桶中。训练过的模型会预测三种不同的鸢尾花品种：山鸢尾（setosa）、维吉尼亚鸢尾（virginica）或者变色鸢尾（versicolor）。

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储 价格](https://cloud.google.com/storage/pricing)，并使用[定价 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade --quiet tensorflow 

### 仅限协作：取消以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 地区

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您需要为每个实例会话创建一个 UUID，并在此教程中创建的资源名称后附加到其上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要做任何操作，因为您已经完成了身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账号授予云存储权限。.

创建一个云存储桶

创建一个存储桶来存储诸如数据集之类的中间结果。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

设定变量

接下来，设定一些在教程中使用的变量。

导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置预构建的容器

为训练和预测设置预先构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 的内存。
     - `n1-highmem`：每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`：每个 vCPU 0.9GB 的内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\] 个CPU核心

*注意：以下机器类型不支持用于训练*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何将一个Python包组装为自定义培训作业。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像操作环境中的指令。

文件 `trainer/task.py` 是用于执行自定义培训作业的Python脚本。*注意*，当我们在工作人员池规范中参考它时，我们将目录斜杠替换为点号 (`trainer.task`) 并删除文件后缀 (`.py`)。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

# Download data
iris_data_filename = 'iris_data.csv'
iris_target_filename = 'iris_target.csv'
data_dir = 'gs://cloud-samples-data/ai-platform/iris'

# gsutil outputs everything to stderr so we need to divert it to stdout.
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_data_filename),
                       iris_data_filename], stderr=sys.stdout)
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_target_filename),
                       iris_target_filename], stderr=sys.stdout)


# Load data into pandas, then use `.values` to get NumPy arrays
iris_data = pd.read_csv(iris_data_filename).values
iris_target = pd.read_csv(iris_target_filename).values

# Convert one-column 2D array into 1D array for use with XGBoost
iris_target = iris_target.reshape((iris_target.size,))


# Load data into DMatrix object
dtrain = xgb.DMatrix(iris_data, label=iris_target)


# Train XGBoost model
bst = xgb.train({}, dtrain, 20)

# Export the classifier to a file
model_filename = 'model.bst'
bst.save_model(model_filename)

# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, model_filename)
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
    stderr=sys.stdout)

请将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

# 训练模型

### [training.create-python-pre-built-container](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)

### [创建 Python 预构建容器](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)

### 创建并运行自定义训练任务

要训练自定义模型，需要执行两个步骤：1）创建自定义训练任务，2）运行任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建自定义训练任务，需要提供以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="iris_" + UUID,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

示例输出：

<google.cloud.aiplatform.training_jobs.CustomTrainingJob object at 0x7feab1346710>

运行自定义训练任务

接下来，通过调用方法 `run` 运行自定义作业来开始训练任务，参数如下：

- `replica_count`: 用于训练的计算实例数量（replica_count = 1 代表单节点训练）。
- `machine_type`: 计算实例的机器类型。
- `base_output_dir`: 用于写入模型工件的 Cloud Storage 位置。
- `sync`: 是否阻塞直到作业完成。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)


job.run(
    replica_count=1, machine_type=TRAIN_COMPUTE, base_output_dir=MODEL_DIR, sync=True
)

MODEL_DIR = MODEL_DIR + "/model"
model_path_to_deploy = MODEL_DIR

### [general.import-model](https://cloud.google.com/vertex-ai/docs/general/import-model)

### [general.import-model](https://cloud.google.com/vertex-ai/docs/general/import-model)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，需要使用以下参数：

- `display_name`：`Model`资源的人类可读名称。
- `artifact`：训练模型 artifacts 的云存储位置。
- `serving_container_image_uri`：服务容器映像。
- `sync`：确定是异步执行上传还是同步执行上传。

如果使用异步执行的`upload()`方法，则可以随后使用`wait()`方法阻塞，等待上传完成。

In [ ]:
model = aip.Model.upload(
    display_name="iris_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

*示例输出：*

    INFO：google.cloud.aiplatform.models：创建模型
    INFO：google.cloud.aiplatform.models：创建模型支撑LRO：projects/759209241365/locations/us-central1/models/925164267982815232/operations/3458372263047331840
    INFO：google.cloud.aiplatform.models：模型已创建。 资源名称：projects/759209241365/locations/us-central1/models/925164267982815232
    INFO：google.cloud.aiplatform.models：在另一个会话中使用此模型：
    INFO：google.cloud.aiplatform.models：model = aiplatform.Model('projects/759209241365/locations/us-central1/models/925164267982815232')

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

制作测试项目

您将使用合成数据作为测试数据项目。不要担心我们使用合成数据 - 我们只是想展示如何进行预测。

In [ ]:
INSTANCES = [[1.4, 1.3, 5.1, 2.8], [1.5, 1.2, 4.7, 2.4]]

### 制作批量输入文件

现在制作一个批量输入文件，你可以将它存储在本地云存储桶中。 预测请求中的每个实例都是以下形式的列表：

                        [ [内容1], [内容2] ]

- `内容`: 测试项的特征值列表。

In [ ]:
import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(str(i) + "\n")

! gsutil cat $gcs_input_uri

### 发送批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法进行批量预测，需要指定以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：存储批量预测结果的Cloud Storage位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测结果的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `sync`：如果设置为True，调用将会在等待异步批处理作业完成时阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="iris_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

示例输出：

    INFO:google.cloud.aiplatform.jobs:创建 BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0> 正在等待上游依赖项完成。
    INFO:google.cloud.aiplatform.jobs:已创建 BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO:google.cloud.aiplatform.jobs:在另一个会话中使用此 BatchPredictionJob：
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO:google.cloud.aiplatform.jobs:查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：
    JobState.JOB_STATE_RUNNING

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

示例输出：

    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob 已创建。资源名称: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO:google.cloud.aiplatform.jobs:在另一个会话中使用此 BatchPredictionJob：
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    INFO:google.cloud.aiplatform.jobs:查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob 运行完成。资源名称: projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的 Cloud 存储输出桶。您可以调用 iter_outputs() 方法获取生成的结果中每个 Cloud 存储文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

*示例输出：*

    {'实例': [1.4, 1.3, 5.1, 2.8]，'预测值': 2.0451931953430176}

## 进行在线预测

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)
### [predictions.deploy-model-api]（https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api）

部署模型

接下来，为在线预测部署您的模型。要部署模型，您需要调用`deploy`方法，使用以下参数：

- `deployed_model_display_name`：部署模型的可读名称。
- `traffic_split`：流量分配给这个模型的百分比，以字典形式指定一个或多个键值对。
如果只有一个模型，则指定为{ "0": 100 }，其中"0"指代上传的这个模型，而100表示100%的流量。
如果端点上存在其他需要分配流量的模型，则使用model_id指定为{ "0": 百分比, model_id: 百分比, ... }，其中model_id是已部署到端点的现有模型的模型id。这些百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `starting_replica_count`：最初预留的计算实例数量。
- `max_replica_count`：可以扩展到的最大计算实例数量。在本教程中，仅预留了一个实例。

In [ ]:
DEPLOYED_NAME = "iris-" + UUID

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

示例输出：

    信息：google.cloud.aiplatform.models:创建终端节点
    信息：google.cloud.aiplatform.models:创建终端节点支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    信息：google.cloud.aiplatform.models:终端节点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:在另一个会话中使用此终端节点：
    信息：google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    信息：google.cloud.aiplatform.models:将模型部署到终端节点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    信息：google.cloud.aiplatform.models:部署终端节点模型支持的LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    信息：google.cloud.aiplatform.models:终端节点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

### [在线预测自动化机器学习](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

制作测试项目

您将使用合成数据作为测试数据项目。不要担心我们使用的是合成数据——我们只是想展示如何进行预测。

In [ ]:
INSTANCE = [1.4, 1.3, 5.1, 2.8]

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源，您可以通过向 `Endpoint` 资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于 predict() 方法可以接受多个项（实例），请将您的单个测试项发送为一个包含一个测试项的列表。

#### 响应

predict() 调用的响应是一个 Python 字典，包括以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类标签的预测置信度，介于0和1之间。
- `deployed_model_id`：进行预测的部署的 `Model` 资源的 Vertex AI 标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

取消模型的部署

当您完成预测时，您可以从“终端点”资源中取消部署模型。这将取消所有计算资源并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI